# Some analysis of the decay data in ICRP Publication 107

This notebook checks the consistency of the <a href="http://www.icrp.org/publication.asp?id=ICRP%20Publication%20107">ICRP Publication 107</a> decay dataset.

### Branching fractions

The branching fractions for the progeny of one radionuclide should be sum to unity for physical reasons. However in practice the sum totals may be slightly different from unity as the branching fractions are only quoted to a maximum of 5 significant figures of precision in the ICRP 107 dataset.

This notebook calculates the sum of progeny branching fractions for each radionuclide in ICRP 107 and analyzes the results.

First read in a CSV file containing the ICRP 107 decay data into a DataFrame.

In [1]:
import pandas as pd
import numpy as np

icrp = pd.read_csv('icrp.csv', index_col='Radionuclide')
icrp.head()

,Element,Z,A,Metastable_state,Half_life,Half_life_units,Half_life_s,Half_life_readable,Num_decay_modes,Mode_1,...,Progeny_1,Mode_2,Fraction_2,Progeny_2,Mode_3,Fraction_3,Progeny_3,Mode_4,Fraction_4,Progeny_4
Radionuclide,,,,,,,,,,,,,,,,,,,,,
Ac-223,Ac,89,223,NaN,2.100,m,1.260000e+02,2.10 m,1,α,...,Fr-219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ac-224,Ac,89,224,NaN,2.780,h,1.000800e+04,2.78 h,2,EC,...,Ra-224,α,0.0910,Fr-220,NaN,NaN,NaN,NaN,NaN,NaN
Ac-225,Ac,89,225,NaN,10.000,d,8.640000e+05,10.0 d,1,α,...,Fr-221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ac-226,Ac,89,226,NaN,29.370,h,1.057320e+05,29.37 h,3,β-,...,Th-226,EC,0.1700,Ra-226,α,0.00006,Fr-222,NaN,NaN,NaN
Ac-227,Ac,89,227,NaN,21.772,y,6.870574e+08,21.772 y,2,β-,...,Th-227,α,0.0138,Fr-223,NaN,NaN,NaN,NaN,NaN,NaN


Find all the radionuclides where the sum of the progeny branching fractions does not equal one and add them to a DataFrame.

In [2]:
nonunity = []
for rn in icrp.index.to_list():
    sum = 0.0
    i = 1
    while True:
        if np.isnan(icrp.loc[rn, 'Fraction_'+str(i)]):
            break
        sum += icrp.loc[rn, 'Fraction_'+str(i)]
        i += 1
        if i > 4: break
    if sum != 1.0:
        nonunity.append([rn, sum])
nonunity = pd.DataFrame(nonunity, columns=['Radionuclide','Total_branching_fraction']).set_index('Radionuclide')

print('There are '+str(len(nonunity))+' radionuclides with a non-unity total for the sum of the branching fractions.')

There are 135 radionuclides with a non-unity total for the sum of the branching fractions.


Sort the DataFrame so that the sum of the branching fractions is increasing.

In [3]:
nonunity.sort_values(by=['Total_branching_fraction'], inplace=True)
nonunity

,Total_branching_fraction
Radionuclide,
At-219,0.970000
U-228,0.975000
Es-254m,0.984410
Es-250,0.985000
Ac-223,0.990000
...,...
Es-255,1.000045
Fr-223,1.000060
Ac-226,1.000060


Considering the radionuclides where the sum of the branching fractions is greater than one first, we see that for all these cases the deviation from unity occurs in the 6<sup>th</sup> significant figure or higher. Therefore these deviations are all consistent with rounding errors given that the branching fractions are recorded in ICRP 107 with up to 5 significant figures of precision.

Cross-checking the radionuclides with the largest sum totals for the branching fractions (Tb-151, Dy-153 etc.) with the original ICRP 107 data file (ICRP-07.NDX), round off errors are clearly the most reasonable explanation. This is because for all these radionuclides there are one or two main decay pathways where the branching fractions add to one, and a minor decay pathway with branching fraction typically of O(10<sup>-5</sup>).

Note the number of radionuclides with total branching fraction greater than one is:

In [4]:
len(nonunity[(nonunity['Total_branching_fraction'] > 1.0)])

81

Now look at the cases where the sum of the branching fractions is less than one and the deviation cannot be assumed to be due to round off errors. The radionuclides for which this applies are:

In [5]:
nonunity[(nonunity['Total_branching_fraction'] < 0.99999)]

,Total_branching_fraction
Radionuclide,
At-219,0.97000
U-228,0.97500
Es-254m,0.98441
Es-250,0.98500
Ac-223,0.99000
Cm-240,0.99700
Po-212m,0.99930
Po-205,0.99940
Np-236,0.99960


Cross-checking the original ICRP 107 data file (ICRP-07.NDX) and another decay data source (<a href="https://www.nndc.bnl.gov/nudat2/chartNuc.jsp">NuDat2</a>), these radionuclides appear to fall into two classes:
1. Round-off of branching fractions in ICRP-07.NDX to the 4<sup>th</sup> or 3<sup>rd</sup> significant figures, which explains the deviation from one (specifically Ag-115, Hf-182m, Np-236, Po-205).
2. Some minor decay pathways not having been included in the ICRP 107 dataset for the other radnionuclides.

The missing minor decay pathways and their approximate branching fractions are:
- Ac-223 electron capture decay (~0.01)
- Am-244m electron capture decay (~0.0004)
- At-217 beta- decay (~0.00012)
- At-219 beta- decay (~0.03)
- Cm-240 electron capture decay (~0.003)
- Es-250 alpha decay (~0.015)
- Es-254m IT decay (~0.01559)
- Po-212m IT decay (~0.0007)
- U-228 electron capture decay (~0.025)

Note that the <a href="https://doi.org/10.11484/jaeri-1347">JAERI 1347</a> and <a href="https://doi.org/10.11484/jaea-data-code-2007-021">JAEA-Data/Code 2007-021</a> reports discuss why these decay pathways were not included in ICRP 107. In short, there were no Evaluated Nuclear Structure Data Files (ENSDF) entries for these decay modes when the ICRP 107 dataset was put together. These reports also confirm that the reason for branching fractions not summing to one for other radionuclides is due to uncertainties in the underlying nuclear data.

### Radioactive progeny not present in ICRP Publication 107

There are some instances where the radioactive progeny resulting from decay pathway that exists in ICRP 107 are not themselves included in ICRP 107. This code finds the affected radionuclides. <a href="https://pyne.io/">PyNE</a> is used to check whether progeny not present in ICRP 107 are radioactive or not.

In [6]:
from pyne import data, nucname

radionuclides = icrp.index.to_list()
missing = {}

for nuc in radionuclides:
    for i in range(1, icrp.loc[nuc, 'Num_decay_modes']+1):
        prog = icrp.loc[nuc, 'Progeny_'+str(i)]

        if prog == 'SF' or prog in radionuclides:
            continue

        hl = data.half_life(nucname.id(prog))
        if hl == float('inf'):
            continue
        
        missing[prog] = "half-life = " + str(hl) + " s, parent is " + nuc

missing

{'Cd-116': 'half-life = 1.0414008e+27 s, parent is Ag-116',
 'Pb-204': 'half-life = 4.418064e+24 s, parent is Tl-204',
 'Xe-124': 'half-life = 5.049216e+21 s, parent is Cs-124',
 'Ba-132': 'half-life = 9.46728e+28 s, parent is La-132m',
 'Xe-134': 'half-life = 1.8303407999999997e+30 s, parent is I-134m',
 'Zn-70': 'half-life = 1.1991888e+26 s, parent is Ga-70',
 'Ta-180m': 'half-life = 2.2405896e+23 s, parent is Hf-180m',
 'Te-128': 'half-life = 2.4299352e+32 s, parent is Sb-128m',
 'Os-184': 'half-life = 1.7672256e+21 s, parent is Pt-188',
 'Ce-138': 'half-life = 2.840184e+21 s, parent is Pr-138m',
 'Ce-142': 'half-life = 1.57788e+24 s, parent is Pr-142',
 'Cr-50': 'half-life = 4.102488e+25 s, parent is V-50',
 'Bi-209': 'half-life = 6.3430776e+26 s, parent is Po-209',
 'Ce-136': 'half-life = 2.209032e+21 s, parent is Pr-136',
 'W-180': 'half-life = 5.680368e+25 s, parent is Ta-180',
 'W-183': 'half-life = 2.1143592e+28 s, parent is Ta-183',
 'Cl-35': 'half-life = nan s, parent is S-3

So there are 17 radioactive progeny resulting from decay pathways in ICRP 107 that are not themselves contained in ICRP 107 (note <sup>35</sup>Cl is actually stable). All of these radionuclides have a half-life that is greater than 10,000 billion years, so can be considered as practically stable. This is probably the reason why they were not included in ICRP 107. Or it may be the case that their radioactivity had not been established when ICRP 107 was put together.